In [23]:
import pandas as pd
import numpy as np



file_path = 'df_pb2.csv'


df= pd.read_csv(file_path, sep=',', low_memory=False)



In [24]:
from IPython.core.display import display, HTML

# Modifier la largeur des cellules dans le notebook
display(HTML("""
<style>
.container {
    width: 100% 
}
</style>
"""))


In [25]:

#df_1000=df
df_1000= df.sample(n=100000, random_state=42)  # random_state pour reproductibilité
df_appartements=df_1000





In [26]:
print(len(df_appartements))
print(df_appartements['N°_département_(BAN)'].head())  # Affiche les 5 premières valeurs de la colonne 'N°_département_(BAN)'


100000
410225    31.0
747505    75.0
195456    31.0
748896    75.0
903676    75.0
Name: N°_département_(BAN), dtype: float64


In [33]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Vérifier que les données de surface habitable ne contiennent pas de valeurs nulles
df_appartements = df_appartements[df_appartements["Surface_habitable_logement"].notna()]

# Convertir les coordonnées projetées en lat/lon (WGS84)
# EPSG:2154 (Lambert 93) vers EPSG:4326 (WGS84)
gdf = gpd.GeoDataFrame(
    df_appartements,
    geometry=[Point(xy) for xy in zip(df_appartements['Coordonnée_cartographique_X_(BAN)'], df_appartements['Coordonnée_cartographique_Y_(BAN)'])],
    crs="EPSG:2154"
)
gdf = gdf.to_crs(epsg=4326)

# Extraire les nouvelles colonnes Latitude et Longitude
df_appartements['Latitude'] = gdf.geometry.y
df_appartements['Longitude'] = gdf.geometry.x
df_appartements = df_appartements.dropna(subset=['Latitude', 'Longitude'])

app = dash.Dash(__name__)

# Mise en page de l'application
# Mise en page de l'application
app.layout = html.Div([
    html.H1("Dashboard Interactif avec Filtres Dynamiques", style={"textAlign": "center"}),

    dcc.Tabs([
        dcc.Tab(label="Carte des Consommations", children=[
            html.Div([
                html.H3("Carte des consommations par localisation", style={"textAlign": "center"}),

                html.Div([
                    html.H4("Choisir la variable pour la couleur des points", style={"color": "black"}),
                    dcc.Dropdown(
                        id="color-variable",
                        options=[
                            {"label": col, "value": col}
                            for col in df_appartements.select_dtypes(include=["float64", "int64"]).columns
                        ],
                        placeholder="Choisir une variable...",
                        style={"width": "100%"},
                    ),

                    html.H4("Filtrer par surface habitable", style={"color": "black"}),
                    dcc.RangeSlider(
                        id="surface-habitable-slider",
                        min=df_appartements["Surface_habitable_logement"].min(),
                        max=df_appartements["Surface_habitable_logement"].max(),
                        step=1,
                        marks={
                            i: f"{i}" for i in range(
                                int(df_appartements["Surface_habitable_logement"].min()),
                                int(df_appartements["Surface_habitable_logement"].max()) + 1,
                                50  # Intervalle des marques tous les 50 m² (ajustable)
                            )
                        },
                        value=[
                            df_appartements["Surface_habitable_logement"].min(),
                            df_appartements["Surface_habitable_logement"].max(),
                        ],
                        tooltip={"placement": "bottom", "always_visible": True}  # Toujours afficher les valeurs sélectionnées
                    ),

                    html.H4("Choisir la variable pour la taille des points", style={"color": "black"}),
                    dcc.Dropdown(
                        id="size-variable",
                        options=[
                            {"label": col, "value": col}
                            for col in df_appartements.select_dtypes(include=["float64", "int64"]).columns
                        ],
                        placeholder="Choisir une variable...",
                        style={"width": "100%"},
                    ),

                    html.H4("Filtrer par Numéro département", style={"color": "black"}),
                    dcc.Dropdown(
                        id="departement-filter",
                        options=[
                            {"label": f" {int(dept)}", "value": dept}
                            for dept in sorted(df_appartements["N°_département_(BAN)"].dropna().unique().astype(int))
                        ],
                        multi=False,
                        placeholder="Choisir un département",
                        style={"width": "100%", "marginBottom": "10px"}
                    ),

                    html.H4("Filtrer par Type de Bâtiment", style={"color": "black"}),
                    dcc.Dropdown(
                        id="type-batiment-filter",
                        options=[
                            {"label": type_b, "value": type_b}
                            for type_b in sorted(df_appartements["Type_bâtiment"].dropna().unique())
                        ],
                        multi=True,
                        placeholder="Choisir un ou plusieurs types de bâtiment...",
                        style={"width": "100%", "marginBottom": "10px"}
                    ),

                    html.H4("Choisir les quantiles pour le filtrage", style={"color": "black"}),
                    dcc.RangeSlider(
                        id="quantile-slider",
                        min=0,
                        max=100,
                        step=1,
                        marks={i: f"{i}%" for i in range(0, 101, 10)},
                        value=[0.5, 90],  # Valeurs par défaut en pourcentage
                    ),
                ], style={"padding": "20px", "backgroundColor": "#EAEAEA"}),

                dcc.Graph(id="carte-consommation"),
            ], style={"padding": "20px"}),
        ])
    ])
], style={"backgroundColor": "#F7F7FF", "color": "black", "fontFamily": "Arial, sans-serif"})


# Callback pour mettre à jour la carte
@app.callback(
    Output("carte-consommation", "figure"),
    [
        Input("color-variable", "value"),
        Input("quantile-slider", "value"),
        Input("surface-habitable-slider", "value"),
        Input("size-variable", "value"),
        Input("departement-filter", "value"),
        Input("type-batiment-filter", "value")
    ],
)
def update_carte(color_var, quantile_range, surface_range, size_var, selected_departement, selected_types_batiment):
    min_surface, max_surface = surface_range

    # Filtrage par surface habitable
    df_filtered = df_appartements[
        (df_appartements["Surface_habitable_logement"] >= min_surface) &
        (df_appartements["Surface_habitable_logement"] <= max_surface)
    ]

    # Filtrer par département
    if selected_departement:
        df_filtered = df_filtered[df_filtered["N°_département_(BAN)"] == selected_departement]

    # Filtrer par type de bâtiment
    if selected_types_batiment:
        df_filtered = df_filtered[df_filtered["Type_bâtiment"].isin(selected_types_batiment)]

    # Calculer la latitude et la longitude moyenne du département sélectionné pour centrer la carte
    if selected_departement:
        dept_coords = df_filtered[df_filtered["N°_département_(BAN)"] == selected_departement][["Latitude", "Longitude"]].mean()
        center_lat = dept_coords["Latitude"]
        center_lon = dept_coords["Longitude"]
    else:
        # Par défaut, centrer sur toute la France
        center_lat = 46.603354
        center_lon = 1.888334

    # Filtrage selon la variable choisie (color_var) et les quantiles dynamiques
    if color_var:
        lower_percentile = quantile_range[0] / 100  # Quantile inférieur
        upper_percentile = quantile_range[1] / 100  # Quantile supérieur
        
        # Calcul des bornes des quantiles
        lower_bound = df_filtered[color_var].quantile(lower_percentile)
        upper_bound = df_filtered[color_var].quantile(upper_percentile)

        # Filtrage des données selon les quantiles
        df_filtered = df_filtered[
            (df_filtered[color_var] >= lower_bound) &
            (df_filtered[color_var] <= upper_bound)
        ]

    # Gestion des cas où aucune donnée ou variable n'est sélectionnée
    if df_filtered.empty:
        return px.scatter_mapbox(
            pd.DataFrame(columns=["Latitude", "Longitude"]),
            lat="Latitude",
            lon="Longitude",
            title="Aucune donnée à afficher. Vérifiez vos filtres.",
        )

    # Vérifier si la variable de taille est sélectionnée, sinon utiliser la surface habitable comme taille par défaut
    if not size_var:
        size_var = "Surface_habitable_logement"  # Utiliser une variable par défaut
    
    # Générer la carte avec une palette de couleurs personnalisée
    fig = px.scatter_mapbox(
        df_filtered,
        lat="Latitude",
        lon="Longitude",
        color=color_var,
        size=size_var,
        hover_name="N°_département_(BAN)",
        hover_data={"Latitude": False, "Longitude": False},
        title="Carte dynamique",
        color_continuous_scale=[
            [0, "red"],   # Vert au début (0%)
            [0.5, "yellow"], 
            [1, "green"]      # Rouge à la fin (100%)
        ],
    )

    # Mise à jour de la mise en page de la carte
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(zoom=5 if not selected_departement else 8, center=dict(lat=center_lat, lon=center_lon)),
        dragmode="zoom",
        height=1000,  # Ajuster la taille de la carte
        width=1600   # Ajuster la taille de la carte
    )

    return fig


if __name__ == "__main__":
    app.run_server(debug=True, port=8060)
